In [1]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-lpgcbvt4
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-lpgcbvt4
  Resolved https://github.com/facebookresearch/detectron2.git to commit 898507047cf441a1e4be7a729270961c401c4354
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 11.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/

In [2]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.0 ; cuda:  2.0.0
detectron2: 0.6


In [3]:
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_train_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.model_zoo import model_zoo
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.structures import BoxMode
from io import BytesIO
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import random
import PIL.Image
import requests
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

In [5]:
#MappingLabelStudio créer une instance à partir du fichier .json générer par LabelStudio
dataset_path = "/kaggle/input/galica"
#dataset_path = "/content/drive/MyDrive/Colab Notebooks/BNF/envoi1/envoi1/dataset"

json_file = dataset_path + "/galica/gallicaimages_set1.json"
image_dir =  dataset_path + "/image"

category_dict = {
  'tampon': 0,
  'écriture manuscrite': 1,
  'écriture typographique': 2,
  'photographie': 3,
  'estampe': 4,
  'décoration' : 5
}

In [6]:
def update_image_sizes(json_file, image_dir, out):
    with open(json_file, encoding='utf-8') as f:
        data = json.load(f)

    for image in data['images']:
        image_id = image['id']
        image_path = image_dir + '/' + image_id + '.jpg'  # Chemin complet de l'image

        # Ouvrir l'image avec PIL
        img = PIL.Image.open(image_path)
        width, height = img.size

        # Mettre à jour les informations de taille dans le JSON
        image['width'] = width
        image['height'] = height

    # Écrire le JSON mis à jour dans un nouveau fichier
    with open(out, 'w') as outfile:
        json.dump(data, outfile)

    print("Mise à jour des tailles d'image terminée.")
    print("JSON mis à jour enregistré dans :", "gallica_dataset_file.json")

In [7]:
#update_image_sizes(json_file, image_dir, "gallica_dataset_file.json")
#update_image_sizes(json_file2, image_dir2, "gallica_dataset_file2.json")
#update_image_sizes(json_file3, image_dir3, "gallica_dataset_file3.json")

In [8]:
gallica_json = "/kaggle/input/gallica-updated/gallica_dataset_file_all.json"
#gallica_json2 = "/kaggle/input/gallica-updated/gallica_dataset_file2.json"
#gallica_json3 = "/kaggle/input/gallica-updated/gallica_dataset_file3.json"

In [9]:
import random

#def get_my_dataset_dicts(json_file, category_dict,target_size = (800, 600)):
def get_my_dataset_dicts(json_file, category_dict, image_dir):
    """
        from gallica Label Studio to COCO dataset json
    """
    with open(json_file, encoding='utf-8') as f:
        data = json.load(f)

    dataset_dicts = []
    for idx, image in enumerate(data['images']):
        record = {}
        # Create file path from local directory and image id
        filename = os.path.join(image_dir, image['id'] + '.jpg')

        # Assign image properties
        record["file_name"] = filename
        record["height"] = image["height"] 
        record["width"] = image["width"]
        record["image_id"] = image['id']

        annotations = []
        # Assign annotations to image
        for ann in data["annotations"]:
            for result in ann["result"]:
                if ann["id"] == image["id"] and type(result["label"]) is not int and result["label"][0] in category_dict.keys():
                # Create a new dict for each annotation in the image
                    x = int(result['bbox']['x'] / 100.0 * image["width"])
                    y = int(result['bbox']['y'] / 100.0 * image["height"])
                    width = int(result['bbox']['width'] / 100.0 * image["width"])
                    height = int(result['bbox']['height'] / 100.0 * image["height"])
                    obj = {
                        "bbox": [x, y, width, height],
                        "bbox_mode": BoxMode.XYWH_ABS,  # as your bounding box coordinates are in absolute format
                        "category_id": category_dict[result["label"][0]],  # map your label name to its corresponding id
#                         "id": result["id"],
#                         "iscrowd": 0,
#                         "image_id":ann["id"],
                    }
                    annotations.append(obj)
        record["annotations"] = annotations
        dataset_dicts.append(record)
    return dataset_dicts

def mix_datasets(dataset1, dataset2, dataset3):
    for data in dataset2:
        dataset1.append(data)
    for data in dataset3:
        dataset1.append(data)
    random.shuffle(dataset1)
    return dataset1

In [10]:

def remove_all_datasets():
    registered_datasets = list(DatasetCatalog.list())
    for dataset_name in registered_datasets:
        DatasetCatalog.remove(dataset_name)
remove_all_datasets()

In [11]:
# PRINT DATASET

def print_coco_dataset(dataset):
    my_dataset_metadata = MetadataCatalog.get(dataset)

    # Get your dataset in Detectron2's format
    dataset_dicts = DatasetCatalog.get(dataset)

    for d in random.sample(dataset_dicts, 3):
        # Open the image file
        img = PIL.Image.open(d["file_name"])
        img = np.array(img)

        # Handle grayscale images:
        if len(img.shape) == 2:
            img = np.stack([img] * 3, axis=-1)

        # Create a visualizer instance
        visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_metadata, scale=0.5)

        # Draw the predictions on the image
        vis = visualizer.draw_dataset_dict(d)

        # Show the image using matplotlib
        plt.imshow(vis.get_image()[:, :, ::-1])
        plt.axis('off')
        plt.show()


In [12]:
def split_data(json_file, category_dict, image_dir, split='train'):
    # Load json file
    """
        Divide original dataset in train , val , test
    """
    with open(json_file) as f:
        data = json.load(f)

    classes_count = np.zeros(len(category_dict))
    # Convert to Detectron2 format
    dataset_dicts = get_my_dataset_dicts(json_file, category_dict, image_dir)
    print(len(dataset_dicts))
    
    #get total number of each classes
    for data in dataset_dicts:        
        for annotation in data["annotations"]:
            classes_count[annotation["category_id"]] += 1
    
    data_train = []
    data_val = []
    data_test = []
    data_lost = []
    nb_image = 0
    count = np.zeros(len(category_dict))
    classes_count_sorted = np.argsort(classes_count)
    for data in dataset_dicts:
        isLost = True
        for annotation in data["annotations"]:
            v = False
            for i in range(len(category_dict)):
                if count[annotation["category_id"]] < classes_count[classes_count_sorted[i]]*0.83:
                    data_train.append(data)
                    for annotation in data["annotations"]:
                        count[annotation["category_id"]] += 1
                    nb_image+=1
                    v=True
                    break
                elif count[annotation["category_id"]] < classes_count[classes_count_sorted[i]]*0.95:
                    data_val.append(data)
                    for annotation in data["annotations"]:
                        count[annotation["category_id"]] += 1
                    nb_image+=1
                    v=True
                    break
                elif count[annotation["category_id"]] < classes_count[classes_count_sorted[i]]:
                    data_test.append(data)
                    for annotation in data["annotations"]:
                        count[annotation["category_id"]] += 1
                    nb_image+=1
                    v=True
                    break
            if v:
                isLost = False
                break
        if isLost:
            data_lost.append(data)
            
    #append lost data to test data
    for data in data_lost:
        data_test.append(data)
                        
    print("count: ", count)
    print("classes_count: ", classes_count)
    print("nb_image: ", nb_image)
    print("data_lost", len(data_lost))
    
    return data_train, data_val, data_test
    
    #print(classes_count)
    #print(np.argsort(classes_count))
datasets = split_data(gallica_json, category_dict, image_dir)

2738
count:  [ 650. 4410. 5598. 1557. 1038.   98.]
classes_count:  [ 650. 4410. 5598. 1557. 1038.   98.]
nb_image:  2718
data_lost 20


In [13]:
def check_repartition(dataset):
    classes_count = np.zeros(len(category_dict))
    for data in dataset:
        for annotation in data["annotations"]:
            classes_count[annotation["category_id"]] += 1
    return classes_count
check_repartition(datasets[2])

array([ 59., 354., 856., 117., 138.,   3.])

In [14]:
import os
import shutil

test_images = []
for i in range(len(datasets[2])):
    test_images.append(datasets[2][i]["image_id"])


# Define source and destination directories
image_dir = dataset_path + "/image"  # Your source directory
destination_dir = "/kaggle/working/test_images"  # Replace with your destination directory

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Iterate through the list of image filenames and copy them to the destination directory
for image_filename in test_images:
    # Add the .jpg extension to the filenames
    image_filename_with_extension = image_filename + ".jpg"
    
    source_path = os.path.join(image_dir, image_filename_with_extension)
    destination_path = os.path.join(destination_dir, image_filename_with_extension)
    
    try:
        shutil.copy(source_path, destination_path)
        print(f"Successfully copied {image_filename} to {destination_dir}")
    except FileNotFoundError:
        print(f"File not found: {image_filename}")
    except FileExistsError:
        print(f"File already exists in the destination directory: {image_filename}")

Successfully copied 2df7c7a8-a476-4827-aed1-d7f613aba42b to /kaggle/working/test_images
Successfully copied 13a8a57e-3017-497f-a3f3-0dd15abdf70f to /kaggle/working/test_images
Successfully copied 6d1b71b9-0477-43d3-b997-050cf3fd79e6 to /kaggle/working/test_images
Successfully copied ca7820d2-679a-4bb2-82b8-a2d5c9c9b2e4 to /kaggle/working/test_images
Successfully copied de439109-a4bf-4b54-93a1-4a7381712bd4 to /kaggle/working/test_images
Successfully copied c7452192-fc03-4614-858e-156a6254a4e3 to /kaggle/working/test_images
Successfully copied d847c2e6-23ab-463b-91f8-c774da7122ab to /kaggle/working/test_images
Successfully copied 41cd740c-935f-4213-9928-c719ae82dd11 to /kaggle/working/test_images
Successfully copied 2de45329-04f6-494d-975a-71302836fe82 to /kaggle/working/test_images
Successfully copied eb0ef928-3735-43a8-adef-4dd218f83c1a to /kaggle/working/test_images
Successfully copied 62fff69c-f976-4d50-96d8-856cc50f99a4 to /kaggle/working/test_images
Successfully copied 43ec59cb-239

In [15]:
zip_file_name = "images.zip"

# Zip the destination directory
shutil.make_archive(destination_dir, 'zip', destination_dir)

# Rename the generated archive to the desired name
os.rename(destination_dir + '.zip', zip_file_name)
print(f"Successfully zipped the directory as {zip_file_name}")

Successfully zipped the directory as images.zip


In [16]:
print(len(datasets[0])/2738*100)
print(len(datasets[1])/2738*100)
print(len(datasets[2])/2738*100)

70.12417823228634
20.4163623082542
9.45945945945946


In [ ]:
random.seed(142)

In [ ]:
def get_dataset_dicts_split(datasets, split='train'):
    # Load json file
    """
        Divide original dataset in train , val , test
    """
    random.seed(42)
    # Shuffle data
    random.shuffle(datasets[0])
    random.shuffle(datasets[1])
    random.shuffle(datasets[2])

    if split == 'train':
        return datasets[0]
    elif split == 'val':
        return datasets[1]
    elif split == 'test':
        return datasets[2]

In [ ]:
#gallica_json = "/kaggle/working/gallica_dataset_file.json"
remove_all_datasets()
for d in ["train", "val", "test"]:
    #DatasetCatalog.register("my_dataset_" + d, lambda d=d: get_dataset_dicts_split(json_file, category_dict, d))
    #MetadataCatalog.get("my_dataset_" + d).set(thing_classes=list(category_dict.keys()))
    DatasetCatalog.register("my_dataset_" + d, lambda d=d: get_dataset_dicts_split(datasets, d))
    MetadataCatalog.get("my_dataset_" + d).set(thing_classes=list(category_dict.keys()))

In [ ]:
print_coco_dataset("my_dataset_val")

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import build_detection_train_loader,  DatasetMapper
from detectron2.data.transforms import ResizeShortestEdge, RandomFlip


In [ ]:
cfg = get_cfg()


# Specify the model to use
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# Override the dataset and solver settings
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS =  model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.000005  # pick a good learning rate
cfg.SOLVER.MAX_ITER = 3000    # adjust up if val mAP is still rising, adjust down if overfit
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 300   # faster, and good enough for this dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(category_dict)  # number of classes in your dataset

In [ ]:
from detectron2.data import transforms as T

In [ ]:
# Spécifiez les augmentations que vous souhaitez utiliser
cfg.INPUT.CROP.ENABLED = True
cfg.INPUT.CROP.SIZE = [0.8, 0.8]

# Utilisez une augmentation standard de Detectron 2 ou définissez la vôtre
augmentation = [
    T.RandomBrightness(0.5, 2.0),
    T.RandomContrast(0.5, 2.0),
    T.RandomSaturation(0.5, 2.0),
    T.RandomRotation(angle=[0, 360], expand=True),
    T.RandomFlip(),
]

cfg.INPUT.AUGMENTATION = augmentation


In [ ]:
import time

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)

# Measure the training time
start_time = time.time()
trainer.train()
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Training time: {:.4f} seconds".format(elapsed_time))

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, COCOPanopticEvaluator
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog

# Get the metadata for your dataset
my_dataset_metadata = MetadataCatalog.get("my_dataset_val")

# Create an evaluator for the validation set
evaluator = COCOEvaluator("my_dataset_val", output_dir="./output")

# Create a test data loader
test_loader = build_detection_test_loader(cfg, "my_dataset_val")

# Perform inference on the validation set
try:
    results = inference_on_dataset(trainer.model, test_loader, evaluator)
except Exception as e:
    # Print the error message
    print("Error:", str(e))
    
    # Get the current image ID
    #img_id = test_loader.dataset.get_img_info(0)["id"]
    # Print the image ID where the error occurred
    #print("Image ID:", img_id)


In [ ]:
from detectron2.utils.visualizer import ColorMode
from detectron2.engine import DefaultPredictor


#Use the final weights generated after successful training for inference  
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8  # set the testing threshold for this model
#Pass the validation dataset
cfg.DATASETS.TEST = ("my_dataset_val", )

predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.checkpoint import DetectionCheckpointer, Checkpointer

checkpointer = DetectionCheckpointer(trainer.model, save_dir=cfg.OUTPUT_DIR)
checkpointer.save("my_model")

In [ ]:
dataset_dicts = DatasetCatalog.get("my_dataset_test")
metadata = MetadataCatalog.get("my_dataset_test")
file_names = ["5f84a46e-a568-462f-8476-35866b41d2ff",
"6a6925ac-5ad8-4829-a88f-0109293e3cd3",
"13a8a57e-3017-497f-a3f3-0dd15abdf70f",
"67a07b78-f7cd-4768-9b12-b65dbc4a396c",
"a5aab339-5d30-405a-bc58-c396bc46002d"]

dataset_test = []
for data in dataset_dicts:
    if data["image_id"] in file_names:
        dataset_test.append(data)

for d in dataset_test:
    im = cv2.imread(d["file_name"])

    # Make predictions
    outputs = predictor(im)

    # Visualize both ground truth and predictions
    visualizer_pred = Visualizer(
        im[:, :, ::-1], metadata=metadata, scale=0.8, instance_mode=ColorMode.IMAGE
    )
    visualizer_gt = Visualizer(
        im[:, :, ::-1], metadata=metadata, scale=0.8, instance_mode=ColorMode.IMAGE
    )


    # Draw ground truth bounding boxes
    gt_visualized = visualizer_gt.draw_dataset_dict(d)

    # Draw instance predictions
    predictions_visualized = visualizer_pred.draw_instance_predictions(outputs["instances"].to("cpu"))


    # Combine the images horizontally
    final_image = np.concatenate((gt_visualized.get_image()[:, :, ::-1], predictions_visualized.get_image()[:, :, ::-1]), axis=1)
    
    # Assuming you have final_image generated
    final_image_bgr = final_image[:, :, ::-1]  # Convert to BGR format

    if not os.path.exists("/kaggle/working/predictions"):
        os.makedirs("/kaggle/working/predictions")
    
    # Save the image to a file (e.g., final_image.png)
    cv2.imwrite('/kaggle/working/predictions/'+d["image_id"]+'.png', final_image_bgr)

    # Show the combined image using matplotlib
    plt.imshow(final_image)
    plt.axis('off')
    plt.show()

In [ ]:
zip_file_name = "predictions.zip"

# Zip the destination directory
shutil.make_archive("/kaggle/working/predictions/", 'zip', "/kaggle/working/predictions/")

# Rename the generated archive to the desired name
os.rename("/kaggle/working/predictions/" + '.zip', zip_file_name)
print(f"Successfully zipped the directory as {zip_file_name}")

In [ ]:
def predict_directory(path):
    n = 0
    for file_name in os.listdir(path):
        if n >= 5:
            break
        # Read the image
        file_path = os.path.join(path, file_name)
        im = cv2.imread(file_path)

        # Measure the prediction time
        start_time = time.time()
        outputs = predictor(im)
        end_time = time.time()
        
        # Calculate the elapsed time
        elapsed_time = end_time - start_time

        print("Inference time: {:.4f} seconds, shape of {}".format(elapsed_time, im.shape))
        
        # Extract the instance masks and their associated classes
        instances = outputs["instances"].to("cpu")
        #instance_masks = instances.pred_masks.numpy()
        print(instances.pred_boxes.tensor.tolist())
        print(instances.pred_classes.tolist())

        # Visualize the predictions
        v = Visualizer(im[:, :, ::-1], metadata=metadata, scale=0.8, instance_mode=ColorMode.IMAGE)
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

        # Display the image
        plt.imshow(v.get_image()[:, :, ::-1])
        plt.axis('off')
        plt.show()
        n+=1
        
predict_directory("/kaggle/input/dataset2/dataset2/1418")

In [ ]:
!python -m pip install coco-pano-ext-demo

In [ ]:
img_size = (400,400)

def create_mask_with_json(data, width, height):
    masks = [np.zeros(img_size, dtype=np.uint8) for x in range(len(category_dict))]
    #mask = np.zeros(img_size, dtype=np.uint8)
    for annotation in data["annotations"]:
        # Convert annotation units to pixel values
        x = int(annotation['bbox'][0])
        y = int(annotation['bbox'][1])
        x2 = int(annotation['bbox'][2])
        y2 = int(annotation['bbox'][3])
        
        # Scale the coordinates to match the mask size
        x_scaled = int(x * img_size[1] / width)      # New x-coordinate after scaling
        y_scaled = int(y * img_size[0] / height)     # New y-coordinate after scaling
        width_scaled = int(x2 * img_size[1] / width)    # New width after scaling
        height_scaled = int(y2 * img_size[0] / height)  # New height after scaling
        

        # Assign the corresponding label value to the pixels within the scaled bounding box
        masks[annotation["category_id"]][y_scaled:y_scaled+height_scaled, x_scaled:x_scaled+width_scaled] = 1
        
    return np.array(masks)

def create_mask_with_prediction(boxes, classes, width, height):
    masks = [np.zeros(img_size, dtype=np.uint8) for x in range(len(category_dict))]
    for i in range(len(boxes)):
        # Convert annotation units to pixel values
        x = int(boxes[i][0])
        y = int(boxes[i][1])
        x2 = int(boxes[i][2])
        y2 = int(boxes[i][3])

        # Scale the coordinates to match the mask size
        x_scaled = int(x * img_size[1] / width)      # New x-coordinate after scaling
        y_scaled = int(y * img_size[0] / height)     # New y-coordinate after scaling
        width_scaled = int(x2 * img_size[1] / width)    # New width after scaling
        height_scaled = int(y2 * img_size[0] / height)  # New height after scaling

        # Assign the corresponding label value to the pixels within the scaled bounding box
        masks[classes[i]][y_scaled:height_scaled, x_scaled:width_scaled] = 1
    
    return np.array(masks)

def predict_image(path):
    # Read the image
    im = cv2.imread(path)

    # Perform prediction
    outputs = predictor(im)

    # Extract the instance masks and their associated classes
    instances = outputs["instances"].to("cpu")

    return (instances.pred_boxes.tensor.tolist(),instances.pred_classes.tolist()) 
        
    
groundtruth = []
predictions = []
for data in dataset_dicts:
    groundtruth.append(create_mask_with_json(data, data["width"], data["height"]))
    prediction = predict_image(data["file_name"])
    predictions.append(create_mask_with_prediction(prediction[0], prediction[1], data["width"], data["height"]))

In [ ]:
np.array(groundtruth).shape

In [ ]:
np.array(groundtruth)[:,1].shape

In [ ]:
np.all(groundtruth[0][1]==0)

In [ ]:

def show_pred(n, class_predicted=1):
    truth = []
    for annotation in dataset_dicts[n]["annotations"]:
        l = annotation['bbox'].copy()
        l[2] += l[0]
        l[3] += l[1]
        truth.append(l)
    print("groundtruth: ", truth)
    # Read the image
    im = cv2.imread(dataset_dicts[n]["file_name"])

    # Perform prediction
    outputs = predictor(im)

    # Extract the instance masks and their associated classes
    instances = outputs["instances"].to("cpu")
    #instance_masks = instances.pred_masks.numpy()
    print("prediction: ", instances.pred_boxes.tensor.tolist())

    # Visualize the predictions
    v = Visualizer(im[:, :, ::-1], metadata=metadata, scale=0.8, instance_mode=ColorMode.IMAGE)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    # Display the image
    plt.imshow(v.get_image()[:, :, ::-1])
    plt.axis('off')
    plt.show()
    
    plt.imshow(groundtruth[n][class_predicted], cmap='binary')  # 'binary' colormap for black and white
    plt.title('Binary Map')
    #plt.colorbar()  # Add a colorbar to show the values
    plt.show()

    plt.imshow(predictions[n][class_predicted], cmap='binary')  # 'binary' colormap for black and white
    plt.title('Binary Map')
    #plt.colorbar()  # Add a colorbar to show the values
    plt.show()

show_pred(0, 2)

In [ ]:
from coco_pano_ext_demo import COCO_plot, COCO
from coco_pano_ext_demo.coco import _compute_labelmap, _compute_iou
from coco_pano_ext_demo.iou import compute_matching_scores

In [ ]:
def compute_matching_weights(target_binary_image, pred_binary_image) -> tuple[list[float], list[float]]:
    # extract connected components
    T = _compute_labelmap(target_binary_image)
    P = _compute_labelmap(pred_binary_image)
    # computes IoUs
    wtp, wpt = 0,0#_compute_iou(T, P)
    if np.all(pred_binary_image==0):
        wtp, wpt = _compute_iou(T, T)
    elif np.all(target_binary_image==0):
        wtp, wpt = _compute_iou(P, P)
    else:
        wtp, wpt = _compute_iou(T, P)
    # remove background components
    wtp, wpt = wtp[1:], wpt[1:]
    return wtp, wpt

In [ ]:
from typing import Iterable
import pandas as pd
def compute_pq_score_list_single_class(targets: Iterable[np.ndarray], predictions: Iterable[np.ndarray]) -> tuple[float, float, float, pd.DataFrame]:
    # Init global accumulators
    W_TtoP_global = []
    W_PtoT_global = []
    # loop over predictions (single class, TODO repeat for each class)
    for T0, P0 in zip(targets, predictions):
        # Compute pairwise matching scores, exluding background
        wtp, wpt = compute_matching_weights(T0, P0)
        # Update global matching lists
        W_TtoP_global.extend(wtp.tolist())
        W_PtoT_global.extend(wpt.tolist())
    # report final score
    pairing_threshold = 0.5
    df = compute_matching_scores(np.array(W_TtoP_global), np.array(W_PtoT_global), pairing_threshold)
    COCO_SQ = df["IoU"].mean() if len(df) > 0 else 0
    COCO_RQ = df["F-score"].iloc[0] if len(df) > 0 else 0
    COCO_PQ = COCO_SQ * COCO_RQ
    
    return COCO_PQ, COCO_RQ, COCO_SQ, df

In [ ]:
#get the average iou of the predictions
scores_per_class = np.zeros(len(category_dict))

for i in range(len(category_dict)):
    T = np.array(groundtruth)[:,i]
    P = np.array(predictions)[:,i]
    scores_per_class[i], COCO_RQ, COCO_SQ, df = compute_pq_score_list_single_class(T, P)

scores_per_class

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Create a DataFrame
df = pd.DataFrame({'Class': list(category_dict.keys()), 'Precision': scores_per_class})

# Remove rows with NaN precision scores
df = df.dropna(subset=['Precision'])

# Create a histogram
plt.bar(df['Class'], df['Precision'])
plt.xlabel('Class')
plt.ylabel('Precision')
plt.title('Precision per Class')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better visibility

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
df

Ram usage : 5-6 GB  
GPU P100 usage : 6GB -> 85% usage  
training time :  10 minutes et 23 secondes  
prediction time : 0.1586 seconds, shape of 3116x3305 (9M de pixels)  
                  0.2412 seconds, shape of 4088x5200 (21M de pixels)